In [1]:
!pip install pycaret optuna pycaret[tuners]

In [2]:
import pandas as pd

# Load heart.csv into a dataframe
heart_data = pd.read_csv("/content/sample_data/heart.csv")

# Load o2Saturation.csv into a dataframe
o2_saturation_data = pd.read_csv("/content/sample_data/o2Saturation.csv")


In [3]:
from pycaret.classification import setup
clf = setup(data=heart_data, target='cp', session_id=123, normalize=True,
            transformation=True)

,Description,Value
0,Session id,123
1,Target,cp
2,Target type,Multiclass
3,Original data shape,"(303, 14)"
4,Transformed data shape,"(303, 14)"
5,Transformed train set shape,"(212, 14)"
6,Transformed test set shape,"(91, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [4]:
from pycaret.classification import setup, compare_models, tune_model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

# Perform feature selection (you can choose a different method)
best_features = compare_models(fold=5, sort='F1')

# Create a custom pipeline with feature selection and Logistic Regression
pipeline = Pipeline([
    ('select', SelectKBest(score_func=f_classif)),
    ('lr', LogisticRegression())
])

# Define a custom grid for hyperparameter tuning
param_grid = {
    'select__k': [1, 2, 3, 4, 5],  # Adjust the number of selected features
    'lr__C': [0.001, 0.01, 0.1, 1, 10],  # Adjust regularization strength
}

# Tune the custom pipeline with the custom grid
tuned_lr = tune_model(pipeline, custom_grid=param_grid)



,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.4953,0.6892,0.4953,0.4725,0.4782,0.2108,0.2132,0.4240
lda,Linear Discriminant Analysis,0.4812,0.6917,0.4812,0.4661,0.4691,0.2067,0.2091,0.1960
nb,Naive Bayes,0.4860,0.6901,0.4860,0.4646,0.4677,0.2286,0.2324,0.3500
rf,Random Forest Classifier,0.5090,0.6910,0.5090,0.4379,0.4638,0.1990,0.2065,1.1420
svm,SVM - Linear Kernel,0.4719,0.0000,0.4719,0.4733,0.4622,0.2169,0.2208,0.3040
ridge,Ridge Classifier,0.4999,0.0000,0.4999,0.4407,0.4615,0.1927,0.1992,0.2760
lr,Logistic Regression,0.4810,0.6877,0.4810,0.4447,0.4578,0.1894,0.1933,1.6840
lightgbm,Light Gradient Boosting Machine,0.4762,0.6733,0.4762,0.4292,0.4481,0.1737,0.1769,0.7180
qda,Quadratic Discriminant Analysis,0.4526,0.6101,0.4526,0.4154,0.4302,0.1351,0.1374,0.6460
gbc,Gradient Boosting Classifier,0.4577,0.6608,0.4577,0.4053,0.4235,0.1361,0.1440,1.1480


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5000,0.6732,0.5000,0.4091,0.4394,0.2244,0.2495
1,0.6364,0.8128,0.6364,0.5227,0.5606,0.4359,0.4847
2,0.5714,0.6772,0.5714,0.4632,0.5014,0.3127,0.3417
3,0.5238,0.7036,0.5238,0.4656,0.4779,0.2473,0.2766
4,0.6667,0.7530,0.6667,0.5159,0.5801,0.4494,0.4811
5,0.5238,0.6641,0.5238,0.5302,0.4794,0.2391,0.2929
6,0.5714,0.7054,0.5714,0.5306,0.5076,0.3415,0.4096
7,0.4762,0.5949,0.4762,0.3730,0.4173,0.1348,0.1433
8,0.4762,0.6601,0.4762,0.4921,0.4337,0.2062,0.2617


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [11]:
from pycaret.classification import setup, create_model, blend_models, stack_models

# Replace 'data' with your actual dataset and target variable
setup(data=heart_data, target='cp')

# Create and train multiple models
model1 = create_model('lr')
model2 = create_model('rf')
model3 = create_model('xgboost')

# Blend the models
blended_model = blend_models(estimator_list=[model1, model2, model3])

# Stack the models
stacked_model = stack_models(estimator_list=[model1, model2], meta_model=model3)


,Description,Value
0,Session id,1092
1,Target,cp
2,Target type,Multiclass
3,Original data shape,"(303, 14)"
4,Transformed data shape,"(303, 14)"
5,Transformed train set shape,"(212, 14)"
6,Transformed test set shape,"(91, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7727,0.8887,0.7727,0.7727,0.7598,0.6417,0.6492
1,0.3182,0.6293,0.3182,0.3091,0.3131,-0.0248,-0.0251
2,0.3810,0.5193,0.3810,0.3803,0.3737,0.0683,0.0697
3,0.3810,0.5355,0.3810,0.3333,0.3524,0.0319,0.0330
4,0.4762,0.7054,0.4762,0.4889,0.4806,0.2143,0.2150
5,0.5714,0.8170,0.5714,0.4796,0.5215,0.3026,0.3093
6,0.5714,0.8616,0.5714,0.5000,0.5284,0.3322,0.3386
7,0.5714,0.7911,0.5714,0.4796,0.5215,0.3077,0.3147
8,0.6667,0.7700,0.6667,0.6073,0.6353,0.4750,0.4808


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5909,0.7784,0.5909,0.5318,0.5409,0.3242,0.3491
1,0.4091,0.6270,0.4091,0.3753,0.3904,0.1118,0.1129
2,0.4762,0.6486,0.4762,0.4794,0.4633,0.2007,0.2088
3,0.5238,0.6104,0.5238,0.3946,0.4493,0.1892,0.2062
4,0.4762,0.6489,0.4762,0.4753,0.4733,0.2007,0.2021
5,0.6190,0.7541,0.6190,0.6222,0.5610,0.3463,0.3805
6,0.7619,0.9397,0.7619,0.6901,0.7171,0.6140,0.6335
7,0.5238,0.7263,0.5238,0.4000,0.4468,0.1860,0.2038
8,0.3810,0.6120,0.3810,0.2721,0.3175,-0.0187,-0.0197


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5455,0.8031,0.5455,0.5070,0.5132,0.2667,0.2744
1,0.4545,0.6529,0.4545,0.4318,0.4416,0.1750,0.1773
2,0.4762,0.6066,0.4762,0.4794,0.4633,0.2007,0.2088
3,0.4286,0.5793,0.4286,0.4240,0.4244,0.1189,0.1193
4,0.4762,0.6833,0.4762,0.5048,0.4837,0.2116,0.2153
5,0.5714,0.7751,0.5714,0.5204,0.5159,0.3077,0.3278
6,0.7143,0.9470,0.7143,0.6429,0.6753,0.5435,0.5530
7,0.5714,0.7703,0.5714,0.4571,0.4987,0.2674,0.2929
8,0.3333,0.6768,0.3333,0.3030,0.3175,-0.0280,-0.0285


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6364,0.7462,0.6364,0.5647,0.5921,0.4114,0.4235
1,0.4545,0.6474,0.4545,0.4373,0.4416,0.1592,0.1621
2,0.5238,0.6224,0.5238,0.5244,0.5166,0.2881,0.2931
3,0.4286,0.6040,0.4286,0.4490,0.4263,0.1711,0.1752
4,0.5714,0.7728,0.5714,0.5941,0.5658,0.3700,0.3815
5,0.5238,0.7553,0.5238,0.4778,0.4960,0.2446,0.2475
6,0.6190,0.8420,0.6190,0.5810,0.5952,0.4207,0.4267
7,0.4762,0.7579,0.4762,0.4011,0.4298,0.1661,0.1709
8,0.3333,0.5605,0.3333,0.3690,0.3462,0.0134,0.0138


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
from pycaret.classification import tune_model

# Tune the model using the 'random' tuner without specifying a search library
tuned_model = tune_model(blended_model, n_iter=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5909,0.8076,0.5909,0.5422,0.5484,0.3333,0.3489
1,0.4091,0.6562,0.4091,0.3778,0.3926,0.0949,0.0960
2,0.5238,0.5979,0.5238,0.5290,0.5010,0.2833,0.3022
3,0.4286,0.5920,0.4286,0.4024,0.4130,0.0903,0.0910
4,0.4762,0.6844,0.4762,0.5048,0.4837,0.2116,0.2153
5,0.5714,0.7774,0.5714,0.5204,0.5159,0.3077,0.3278
6,0.7619,0.9426,0.7619,0.6825,0.7186,0.6196,0.6304
7,0.5714,0.7891,0.5714,0.4571,0.4987,0.2674,0.2929
8,0.3810,0.6940,0.3810,0.3175,0.3463,0.0215,0.0219


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [13]:
from pycaret.classification import evaluate_model

evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
!pip install streamlit

In [15]:
import streamlit as st
import pandas as pd
from pycaret.classification import load_model, predict_model

In [21]:
# Load the saved model from your Google Drive (or the directory where you saved it)
model = load_model('/content/sample_data/model.pkl')

FileNotFoundError: ignored

In [25]:
%%writefile prediction_app.py

st.title('Your Model Prediction App')

# Collect user input
input_1 = st.number_input('Input 1', min_value=0.0, max_value=10.0)
input_2 = st.number_input('Input 2', min_value=0.0, max_value=10.0)
input_3 = st.number_input('Input 3', min_value=0.0, max_value=10.0)
input_4 = st.number_input('Input 4', min_value=0.0, max_value=10.0)

# Predict the output
if st.button('Predict'):
  input_data = pd.DataFrame([[input_1, input_2, input_3, input_4]],
  columns=['input_1', 'input_2', 'input_3', 'input_4'])
  prediction = predict_model(model, data=input_data)
  st.write(f"The predicted output is: {prediction['Label'].iloc[0]}")

Writing prediction_app.py


In [26]:
!streamlit run prediction_app.py





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.233.208:8501

  Stopping...
  Stopping...
